# Scheduled Task Creation
Detects the creation of scheduled tasks in user session

## Rule Content
```
- title: Scheduled Task Creation
  id: 92626ddd-662c-49e3-ac59-f6535f12d189
  status: experimental
  description: Detects the creation of scheduled tasks in user session
  author: Florian Roth
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image: '*\schtasks.exe'
      CommandLine: '* /create *'
    filter:
      User: NT AUTHORITY\SYSTEM
    condition: selection and not filter
  fields:
  - CommandLine
  - ParentCommandLine
  tags:
  - attack.execution
  - attack.persistence
  - attack.privilege_escalation
  - attack.t1053
  - attack.s0111
  - car.2013-08-001
  falsepositives:
  - Administrative activity
  - Software installation
  level: low

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:*\\schtasks.exe AND process_command_line.keyword:*\ \/create\ *) AND (NOT (user_account:"NT\ AUTHORITY\\SYSTEM")))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()